In [ ]:
# Import Libs
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
# Functions to generate datasets


def generate_linear(n=100):
    import numpy as np
    pts = np.random.uniform(0, 1, (n, 2))
    inputs = []
    labels = []
    for pt in pts:
        inputs.append([pt[0], pt[1]])
        distance = (pt[0] - pt[1]) / 1.414
        if pt[0] > pt[1]:
            labels.append(0)
        else:
            labels.append(1)
    return np.array(inputs), np.array(labels).reshape(n, 1)


def generate_XOR_easy():
    import numpy as np
    inputs = []
    labels = []

    for i in range(11):
        inputs.append([0.1 * i, 0.1 * i])
        labels.append(0)

        if 0.1 * i == 0.5:
            continue

        inputs.append([0.1 * i, 1 - 0.1 * i])
        labels.append(1)

    return np.array(inputs), np.array(labels).reshape(21, 1)

In [13]:
# Plot results

def show_result(x,y,pred_y):
    import matplotlib.pyplot as plt
    plt.subplot(1,2,1)
    plt.title('Ground Truth',fontsize=18)
    for i in range(x.shape[0]):
        if y[i] == 0:
            plt.plot(x[i][0],x[i][1],'ro')
        else:
            plt.plot(x[i][0],x[i][1],'bo')
    plt.subplot(1,2,2)
    plt.title('Predict result',fontdict=18)

    for i in range(x.shape[0]):
        if pred_y[i] == 0:
            plt.plot(x[i][0],x[i][1],'ro')
        else:
            plt.plot(x[i][0],x[i][1],'bo')

    plt.show()


In [ ]:
# Activative functions and their derivatives
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def derivative_sigmoid(x):
    return np.multiply(x, 1.0 - x)


x = np.random.uniform(0, 1, (10, 1))

# Function mapping
activation_map = {
    "sigmoid": sigmoid
}

[[0.34256061]
 [0.95085028]
 [0.14740149]
 [0.11402011]
 [0.28265281]
 [0.11480027]
 [0.55311621]
 [0.06753801]
 [0.08898468]
 [0.68894206]]
[[0.58481239]
 [0.72128614]
 [0.5367838 ]
 [0.52847419]
 [0.57019648]
 [0.52866859]
 [0.63485827]
 [0.51687809]
 [0.5222315 ]
 [0.66573154]]


In [ ]:
class Linear_Layer:

    def __init__(self, input_size, output_size,activation = "sigmoid"):

        # we should check the activation function is in the activation_map
        # we use sigmoid as default activation function
        if activation not in activation_map:
            activation = "sigmoid"

        self.input_size = input_size
        self.output_size = output_size
        # Initialize weights and bias (W , b)
        # Assume that the size of input vector is  n by 1
        # The size of weight matrix should be m by n, where m is the size of output vector

        # According to the guide that TA provided, the initial weights often use a small random value

        self.weights = np.random.randn(output_size, input_size) * 0.01
        self.bias = np.zeros((output_size, 1))
        self.input = None

    #   Forward pass: y = Wx + b
    #   Ｒegular training process
    def forward(self, x):
        self.input = x
        return self.activation(np.matmul(self.weights, x) + self.bias)

    def backward(self, grad_output):
        grad_input = np.dot(grad_output, self.weights.T)
        grad_weights = np.dot(self.input.T, grad_output)
        grad_bias = np.sum(grad_output, axis=0, keepdims=True)

        # Update weights and bias
        self.weights -= 0.01 * grad_weights
        self.bias -= 0.01 * grad_bias

        return grad_input


[[ 0.54510296 -0.51411106]
 [ 0.11259701  0.09565303]]
